In [1]:
## Here we will write out the matrix elements for our non-eq setup

from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io

In [2]:
## Firstly, we have to define the integral function C and D

def integral1(i,k,tb,beta,mu,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,gamma),limit=limit_value)[0]
    return integral

def integral2(i,k,tb,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma),limit=limit_value)[0]
    return integral

def C(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + 0.5*(func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))

    return val

def D(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + integral2(i,k,tb,gamma,eigenergies) + 0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))
    return val

In [4]:
NL1 = 1
NL2 = 1
NM = 2

N = NL1 + NL2 + NM
dL1 = 2**NL1
dL2 = 2**NL2
dM = 2**NM
d = 2**N
dims = [2]*N

In [ ]:
NL1 = 2
NL2 = 2
NM = 2

N = NL1 + NL2 + NM
dL1 = 2**NL1
dL2 = 2**NL2
dM = 2**NM
d = 2**N
dims = [2]*N

In [5]:
## Define the Hamiltonian 

w0list = np.linspace(1,1,N)
glist = np.linspace(0.0016,0.0016,N-1)

delta = 1

H_S = create_hamiltonian(w0list,glist,delta,N)

create_sm_list_left = [create_sm(N,i + 1) for i in range(NL1)]
create_sm_list_right = [create_sm(N,NM + NL1 + i + 1) for i in range(NL2)]  # Create list of sigma_- operators

eigenergies,eigstates=H_S.eigenstates()

In [78]:
## Let us compute the thermal density matrix
beta1 = 1  #left baths
beta2 = 1.6 #right baths

rho_th = (-beta1*H_S).expm()/((-beta1*H_S).expm()).tr() #left thermal density matrix
#print(rho_th)

In [5]:
## Now we set parameters for the  non-eq setup

beta_list_left = np.linspace(beta1,beta1,NL1)
beta_list_right = np.linspace(beta2,beta2,NL2)

mu_list_left = np.linspace(0,0,NL1)
mu_list_right = np.linspace(0,0,NL2)

gamma_list_left = np.linspace(1,1,NL1)  #coupling strengths to left baths
gamma_list_right = np.linspace(1,1,NL2)  #coupling strengths to right baths

tb = 0.01

In [39]:
print(gamma_list_left)

[1.]


In [44]:
number = len(eigenergies)

In [100]:
gamma1 = 1
gamma2 = 1

beta2 = beta_list_right[0]
mu2 = mu_list_right[0]

beta1 = beta_list_left[0]
mu1 = mu_list_left[0]

limit_value = 700
b = 500

In [101]:
integral11=np.empty((number,number),dtype=np.cdouble) #stores J * N integral for left bath
integral12=np.empty((number,number),dtype=np.cdouble) # stores J integral (just to check) for the left bath
integral21=np.empty((number,number),dtype=np.cdouble) #stores J*N integral for right bath
integral22=np.empty((number,number),dtype=np.cdouble)

        #print("Integral calculations at beta2 = {} and e = {} are : ".format(beta2,e))

for i in range(number):
    for k in range(number):
                freq=eigenergies[k]-eigenergies[i]
                #print(f"Absolute frequency  for i = {i}, k = {k} is ",np.absolute(freq))
                #print(i,k,freq)
                if( np.absolute(freq) >= 1/10**10):
                    integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta2,mu2,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                    integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #left bath done
                    integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                    integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #right bath
        
                if (np.absolute(freq)<=1/10**10):  #The problem is arising here....
                    integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta2,mu2,gamma1),limit=limit_value)[0]
                    integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma1),limit=limit_value)[0]
                    integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value)[0]
                    integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma2),limit=limit_value)[0]
                
            
            #expected=1.0j*(eigenergies[k]-eigenergies[i])/(2*tb*tb)
        #        print(i,k,integral2[i,k],expected)
    
    
        # PAY ATTENTION TO THE WAY THESE COEFFICIENTS ARE BEING COMPUTED
    
constant12=np.empty((number,number),dtype=np.cdouble)
constant11=np.empty((number,number),dtype=np.cdouble)
constant21=np.empty((number,number),dtype=np.cdouble)
constant22=np.empty((number,number),dtype=np.cdouble)
        
        
        
for i in range(number):
        for k in range(number):
                constant12[i,k]=integral12[i,k]+integral11[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma1)+func1(eigenergies[k]-eigenergies[i],tb,beta2,mu2,gamma1))    #full coefficient created this is nbar+1
                constant11[i,k]=integral11[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta2,mu2,gamma1)                                       # the full coefficient is created
                
                constant22[i,k]=integral22[i,k]+integral21[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma2)+func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2))    #full coefficient created this is nbar+1
                constant21[i,k]=integral21[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2)   # the full coefficient is created
                #print(i,k,constant11[i,k],constant12[i,k],constant21[i,k],constant22[i,k])

c:\Users\DELL\OneDrive\Desktop\Term paper and Proj report\ICTS Project\SearchingLindbladians\python\helper_code_qutip.py:205: RuntimeWarning: divide by zero encountered in scalar divide
  return 1/(np.exp(beta*(omega-mu))-1)
C:\Users\DELL\AppData\Local\Temp\ipykernel_51368\1308690218.py:20: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta2,mu2,gamma1),limit=limit_value)[0]
C:\Users\DELL\AppData\Local\Temp\ipykernel_51368\1308690218.py:22: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value)[0]


In [45]:

epsilon = 0.01

## Now we will write out the matrix elements

A = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        sum = 0
        vi = eigstates[i]
        vk = eigstates[k]
        proj_i = vi*vi.dag()
        proj_k = vk*vk.dag()
        for y in range(number):
            for l in range(NL1):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_left[l]*proj_y,create_sm_list_left[l].dag())*constant11[k,y]
                sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_left[l].dag(),proj_y*create_sm_list_left[l]*proj_k)*constant12[y,k]
                sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

            for l in range(NL2):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_right[l]*proj_y,create_sm_list_right[l].dag())*constant21[k,y]
                sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_right[l].dag(),proj_y*create_sm_list_right[l]*proj_k)*constant22[y,k]
                sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

        A[i,k] = sum


In [42]:
## Let us form a liouvillian for this redfield and check it's steady state

c_1 = create_sm(N,1)
c_N = create_sm(N,N)

pre=-1.0j*H_S
post=1.0j*H_S
        
L=spre(pre)+spost(post)
        
for i in range(number):
    for k in range(number):
        vi=eigstates[i]
        vk=eigstates[k]

        #print(constant11[i,k],constant12[i,k])
                
        op1=epsilon*epsilon*C(i,k,tb,beta_list_left[0],mu_list_left[0],gamma_list_left[0],eigenergies)*vi*vi.dag()*c_1*vk*vk.dag()*c_1.dag()
        op2=epsilon*epsilon*D(i,k,tb,beta_list_left[0],mu_list_left[0],gamma_list_left[0],eigenergies)*c_1.dag()*vi*vi.dag()*c_1*vk*vk.dag()
                
        op3=epsilon*epsilon*C(i,k,tb,beta_list_left[0],mu_list_left[0],gamma_list_left[0],eigenergies)*c_1.dag()
        op4=vi*vi.dag()*c_1*vk*vk.dag()
        op5=epsilon*epsilon*D(i,k,tb,beta_list_left[0],mu_list_left[0],gamma_list_left[0],eigenergies)*c_1.dag()
                
                
        L=L+spre(-op2-op1.dag())+spost(-op1-op2.dag())
        L=L+spre(op3)*spost(op4)+spre(op4)*spost(op5)+spre(op4.dag())*spost(op3.dag()) +spre(op5.dag())*spost(op4.dag())
                
        op1=epsilon*epsilon*C(i,k,tb,beta_list_right[0],mu_list_right[0],gamma_list_right[0],eigenergies)*vi*vi.dag()*c_N*vk*vk.dag()*c_N.dag()
        op2=epsilon*epsilon*D(i,k,tb,beta_list_right[0],mu_list_right[0],gamma_list_right[0],eigenergies)*c_N.dag()*vi*vi.dag()*c_N*vk*vk.dag()
                
        op3=epsilon*epsilon*C(i,k,tb,beta_list_right[0],mu_list_right[0],gamma_list_right[0],eigenergies)*c_N.dag()
        op4=vi*vi.dag()*c_N*vk*vk.dag()
        op5=epsilon*epsilon*D(i,k,tb,beta_list_right[0],mu_list_right[0],gamma_list_right[0],eigenergies)*c_N.dag()
                
                
        L=L+spre(-op2-op1.dag())+spost(-op1-op2.dag())
        L=L+spre(op3)*spost(op4)+spre(op4)*spost(op5)+spre(op4.dag())*spost(op3.dag()) +spre(op5.dag())*spost(op4.dag())

return_info=True
        #print('Redfield Liouvillian constructed, Computing steady-state ...')
ss_redfield = steadystate(L,return_info=return_info)
L_eigen = L.eigenenergies()
print("Smallest eigenvalues of L_red are ", L_eigen[-3:])
print(tracedist(rho_th,ss_redfield))


C:\Users\DELL\AppData\Local\Temp\ipykernel_51368\3743586826.py:8: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,gamma),limit=limit_value)[0]
d:\Anaconda\envs\qiskitenv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\Anaconda\envs\qiskitenv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: array must not contain infs or NaNs

In [79]:
L=spre(pre)+spost(post)
        
for i in range(number):
    for k in range(number):
        vi=eigstates[i]
        vk=eigstates[k]

        #print(constant11[i,k],constant12[i,k])
                
        op1=epsilon*epsilon*constant11[i,k]*vi*vi.dag()*c_1*vk*vk.dag()*c_1.dag()
        op2=epsilon*epsilon*constant12[i,k]*c_1.dag()*vi*vi.dag()*c_1*vk*vk.dag()
                
        op3=epsilon*epsilon*constant11[i,k]*c_1.dag()
        op4=vi*vi.dag()*c_1*vk*vk.dag()
        op5=epsilon*epsilon*constant12[i,k]*c_1.dag()
                
                
        L=L+spre(-op2-op1.dag())+spost(-op1-op2.dag())
        L=L+spre(op3)*spost(op4)+spre(op4)*spost(op5)+spre(op4.dag())*spost(op3.dag()) +spre(op5.dag())*spost(op4.dag())
                
        op1=epsilon*epsilon*constant21[i,k]*vi*vi.dag()*c_N*vk*vk.dag()*c_N.dag()
        op2=epsilon*epsilon*constant22[i,k]*c_N.dag()*vi*vi.dag()*c_N*vk*vk.dag()
                
        op3=epsilon*epsilon*constant21[i,k]*c_N.dag()
        op4=vi*vi.dag()*c_N*vk*vk.dag()
        op5=epsilon*epsilon*constant22[i,k]*c_N.dag()
                
                
        L=L+spre(-op2-op1.dag())+spost(-op1-op2.dag())
        L=L+spre(op3)*spost(op4)+spre(op4)*spost(op5)+spre(op4.dag())*spost(op3.dag()) +spre(op5.dag())*spost(op4.dag())

return_info=True
        #print('Redfield Liouvillian constructed, Computing steady-state ...')
ss_redfield = steadystate(L,return_info=return_info)
L_eigen = L.eigenenergies()
print("Smallest eigenvalues of L_red are ", L_eigen[-3:])
print(tracedist(rho_th,ss_redfield))

Smallest eigenvalues of L_red are  [-5.42389319e-05+9.99929106e-01j -5.42389318e-05-9.99929106e-01j
 -2.47758562e-16+8.71429137e-16j]
3.142788891137614e-05


In [80]:
rho_th_new = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        vi = eigstates[i]
        vk = eigstates[k]

        rho_th_new[i,k] = vi.dag()*rho_th*vk

#print(rho_th_new)

rho_th_diag = [rho_th_new[i,i] for i in range(number)]
print(rho_th_diag)


[(0.2867104807443116+0j), (0.1054748914342129+0j), (0.10527736219305431+0j), (0.10480200765390596+0j), (0.10432879946353746+0j), (0.0388020441184368+0j), (0.03872937717158434+0j), (0.038644926216399096+0j), (0.03855450400936415+0j), (0.038380420444733665+0j), (0.03821890796739024+0j), (0.014274474306600175+0j), (0.014247741630800463+0j), (0.014183409389607018+0j), (0.014119367625133614+0j), (0.005251285630928183+0j)]


In [47]:
print(rho_th_new)

[[ 2.86710481e-01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  1.05474891e-01+0.j -8.20177259e-15+0.j
  -3.22658567e-16+0.j -2.60208521e-17+0.j  1.00211849e-18+0.j
   4.42983084e-18+0.j -2.79480471e-18+0.j -1.02536577e-18+0.j
   2.79289411e-18+0.j -2.38088094e-18+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -8.19483370e-15+0.j  1.05277362e-01+0.j
  -2.66453526e-15+0.j  9.85322934e-16+0.j  1.59959941e-18+0.j
   5.88321269e-18+0.j -5.06369168e-18+0.j -6.42872337e-18+0.j
   3.30436140e-18+0.j -2.92655704e-18+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 

In [81]:
## Now we use the lin algebra solver to solve the equation Ax = b, where b is all zeroes

b = np.zeros((number),dtype=complex)

x = np.linalg.solve(A,b)

print(x)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [49]:
print(A)

[[ 1.15015514e-04+0.00000000e+00j -7.83117888e-05+0.00000000e+00j
  -1.33786343e-04+0.00000000e+00j -7.85113709e-05+0.00000000e+00j
  -2.30368163e-05+0.00000000e+00j -1.12717609e-52+0.00000000e+00j
  -2.40890951e-36+0.00000000e+00j -1.71413653e-37+0.00000000e+00j
  -7.28212805e-37+0.00000000e+00j -5.62780810e-52+0.00000000e+00j
  -2.47363129e-52+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [-2.88092971e-05+3.17637355e-22j  1.64628843e-04-5.05572790e-21j
   4.10116148e-21+3.21276950e-21j  1.41745670e-19-3.24254800e-21j
  -3.13666888e-21+5.02925812e-22j -1.17467683e-04+1.69406589e-21j
  -6.68931716e-05+2.96461532e-21j -2.13267719e-21+2.11758237e-22j
  -3.92556854e-05-8.47032947e-22j -1.15184081e-05+1.90582413e-21j
   1.86745568e-21+7.94093388e-22j -1.30897140e-37+7.48888019e-53j
  -2.39511523e-36-1.51437770e-52j -4.26058697e-36+1.49078548e-53j
  -2.6654

In [85]:
## Now we can add one more constraint, that the sum of all the elements of x should be 1, which is basically another row in the matrix A consisting of all 1

#A = np.vstack([A,np.ones((1,number))])

#print(A)

#print(np.linalg.matrix_rank(A))

q,r = np.linalg.qr(A.T)

print(r)
diag_r = [r[i][i] for i in range(number)]

#print(np.linalg.matrix_rank(A))





[[-2.09654050e-04+0.00000000e+00j  7.72982697e-05-1.17156199e-21j
   1.54390087e-04-4.42800094e-21j  7.75105470e-05+8.97693915e-22j
   1.88676631e-05+2.32118846e-22j -2.33387116e-05+1.43055995e-22j
  -1.99927653e-05+9.87787874e-23j -2.41810099e-05-1.79038301e-22j
  -2.68680227e-05+2.69716444e-23j -1.23248225e-05+1.59367111e-22j
  -1.17071844e-05-1.68710946e-22j -5.64879561e-37-1.71367445e-53j
  -3.79732383e-37+3.52303280e-55j -5.95177811e-37+2.75218790e-54j
  -4.60780281e-37-1.04795352e-53j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j -2.04706240e-04+0.00000000e+00j
   2.35642409e-05+2.43127155e-21j -1.21997675e-05-6.17756594e-22j
  -1.12298905e-05+1.21076861e-21j  1.48821082e-04+1.66446563e-21j
   8.21419271e-05+7.66105230e-21j -9.13089032e-06-1.46359383e-21j
   4.25383549e-05+1.34244009e-21j  1.07841792e-05+5.13370384e-22j
  -4.42070107e-06+8.83574037e-22j -3.58542227e-05-7.29154769e-22j
  -1.02379982e-05-1.01055903e-22j -1.37587154e-05-4.49388845e-22j
  -6.3113

In [86]:
print(diag_r)

[(-0.00020965405007135536+0j), (-0.00020470623984280954+0j), (-0.0001661701352959628+0j), (-0.00018223416588184914+0j), (-0.00019698446985397404+0j), (-0.00019500467484584056+0j), (-0.00020378989201450085+0j), (-0.00020034420100734356+0j), (-0.00019547323665119104+0j), (-0.00019841190552281565+0j), (-0.0002022180508738586+0j), (-0.00017180579979412605+0j), (-0.00022895695445542598+0j), (-0.00022206817798819524+0j), (-0.00013060322338152944+0j), (5.705136391556217e-20+0j)]


In [88]:
## Since last element of r is 0, we can remove the last row of  A and add a row of 1's

A_new = A[:-1]
A_new = np.vstack([A_new,np.ones((1,number))])

#print(np.linalg.matrix_rank(A_new))
print(A_new.shape)
#print(A_new)

(16, 16)


In [89]:
print(A.shape)

(16, 16)


In [90]:
print(np.linalg.matrix_rank(A))
print(A.shape)

15
(16, 16)


In [91]:

print(x)b[-1] = 1  ## Last element of b is 1

x = np.linalg.solve(A_new,b)

[0.28671048+3.45684540e-19j 0.10547489-4.72333595e-19j
 0.10527736+3.65160193e-18j 0.10480201-3.98372149e-18j
 0.1043288 -4.29827884e-18j 0.03880204-1.02120690e-18j
 0.03872938-1.85176045e-18j 0.03864493+1.09135028e-18j
 0.0385545 +6.95396895e-19j 0.03838042-2.53621039e-19j
 0.03821891-2.19906321e-18j 0.01427447+1.13773746e-18j
 0.01424774+2.08960827e-18j 0.01418341+1.56500138e-18j
 0.01411937+1.35538897e-19j 0.00525129+3.36806586e-18j]


In [92]:
##Let us check if this is correct

print(np.dot(A_new,x))
print(np.dot(A[-1],x))

[-1.35525272e-20+0.00000000e+00j  3.38813179e-21+0.00000000e+00j
 -8.47032947e-21+1.41059322e-37j  6.77626358e-21+1.41059322e-37j
  0.00000000e+00-1.05794492e-37j -2.11758237e-22-4.70197740e-38j
 -8.47032947e-22-7.05296610e-38j  8.47032947e-22+9.40395481e-38j
  8.47032947e-22+0.00000000e+00j -1.69406589e-21+4.70197740e-38j
  4.23516474e-22+2.35098870e-38j -2.11758237e-22+1.29304379e-37j
  4.23516474e-22+0.00000000e+00j -4.23516474e-22-1.29304379e-37j
 -8.47032947e-22+1.17549435e-38j  1.00000000e+00-1.15555797e-33j]
(1.1117307432712692e-20+8.71620651727874e-22j)


In [93]:
#We have to tranfer the basis of the solution rho matrix to computational basis from the eigenbasis

x_real = [np.real(x[i]) for i in range(number)]

rho = np.diag(x_real)

#set U matrix whose columns are the eigenvectors of the Hamiltonian

U = np.zeros((number,number),dtype=complex)
for i in range(number):
    U[:,i] = eigstates[i].full().flatten()

check = 1

print(U[:,check])
print(eigstates[check])

[ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
 -5.81313884e-17+0.j  0.00000000e+00+0.j -3.19375396e-22+0.j
 -3.48961025e-17+0.j -5.00000000e-01+0.j  0.00000000e+00+0.j
  9.29267280e-17+0.j -8.00716279e-17+0.j -5.00000000e-01+0.j
  1.66825355e-17+0.j -5.00000000e-01+0.j -5.00000000e-01+0.j
  0.00000000e+00+0.j]
Quantum object: dims=[[2, 2, 2, 2], [1, 1, 1, 1]], shape=(16, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-5.81313884e-17]
 [ 0.00000000e+00]
 [-3.19375396e-22]
 [-3.48961025e-17]
 [-5.00000000e-01]
 [ 0.00000000e+00]
 [ 9.29267280e-17]
 [-8.00716279e-17]
 [-5.00000000e-01]
 [ 1.66825355e-17]
 [-5.00000000e-01]
 [-5.00000000e-01]
 [ 0.00000000e+00]]


In [58]:
print([x.real for x in  rho_th_diag])
print(x_real)

[0.2867104807443116, 0.1054748914342129, 0.10527736219305431, 0.10480200765390596, 0.10432879946353746, 0.0388020441184368, 0.03872937717158434, 0.038644926216399096, 0.03855450400936415, 0.038380420444733665, 0.03821890796739024, 0.014274474306600175, 0.014247741630800463, 0.014183409389607018, 0.014119367625133614, 0.005251285630928183]
[0.2867104807443114, 0.10547489143421288, 0.10527736219305421, 0.10480200765390596, 0.1043287994635375, 0.038802044118436796, 0.03872937717158437, 0.03864492621639911, 0.03855450400936417, 0.038380420444733665, 0.03821890796739025, 0.014274474306600197, 0.014247741630800498, 0.014183409389607032, 0.014119367625133621, 0.0052512856309282294]


In [76]:
rho_comp = np.dot(np.conjugate(U.T),np.dot(rho,U))
rho_comp2 = np.dot(U,np.dot(rho,U.T.conjugate()))

#print(rho_comp)
print(np.trace(rho_comp))
print(np.trace(rho_comp2))

(0.999999999999999+0j)
(0.9999999999999999+0j)


In [2]:
def L2_red(rho,eigstates):
    sum = 0
    rho = Qobj(rho)
    rho.dims = [dims,dims]
    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()

            for l in range(NL1):
                op = commutator(rho*proj_i*create_sm_list_left[l]*proj_k,create_sm_list_left[l].dag())*constant11[i,k] + commutator(create_sm_list_left[l].dag(),proj_i*create_sm_list_left[l]*proj_k*rho)*constant12[i,k]
                sum += op
                sum += op.dag()

            for l in range(NL2):
                op = commutator(rho*proj_i*create_sm_list_right[l]*proj_k,create_sm_list_right[l].dag())*constant21[i,k] + commutator(create_sm_list_right[l].dag(),proj_i*create_sm_list_right[l]*proj_k*rho)*constant22[i,k]
                sum += op
                sum += op.dag()
    data = sum.full()
    sum = np.array(data,dtype=complex)
    return sum

In [95]:
L2_redfield = L2_red(rho_comp2,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb)

In [96]:
rho_th_qutip = Qobj(rho_th)
rho_th_qutip.dims = [dims,dims]

rho_comp_qutip = Qobj(rho_comp2)
rho_comp_qutip.dims = [dims,dims]
print(tracedist(rho_th_qutip,rho_comp_qutip))

9.325898992962887e-15


In [70]:
print(L2_redfield)

[[-1.75424937e-02+0.00000000e+00j  3.42594084e-06-2.70922976e-06j
   3.43865866e-03+1.79189227e-06j  5.59431516e-08-4.44299356e-08j
  -4.79311394e-09+1.58183376e-08j  3.90909155e-11-1.29446245e-10j
   9.77990678e-15+2.04806870e-15j -1.00942570e-21-4.98944079e-23j
   3.84861286e-11-9.96900003e-11j -3.39670128e-17-1.99926671e-15j
   2.08932799e-20-1.20215605e-17j  1.31599066e-21-3.37893244e-22j
   3.23468727e-20-4.32379111e-20j  4.96056982e-22-2.61117911e-23j
  -8.05566515e-22+3.71181652e-22j  0.00000000e+00+0.00000000e+00j]
 [ 3.42594084e-06+2.70922976e-06j  1.12642431e-02+0.00000000e+00j
   1.88680727e-02+1.23924022e-02j -1.30097094e-02+4.26024326e-06j
  -8.66949584e-03-5.08217773e-03j  2.27428491e-09+3.53464596e-09j
  -5.14400113e-09-1.27673891e-08j  1.57466229e-14-4.01584333e-13j
   1.83728219e-06-1.46175489e-06j -2.35805388e-09+7.80916566e-09j
   9.65938493e-13+2.37193931e-11j -2.99217647e-19-1.31693609e-21j
  -1.95156391e-18+1.11198485e-17j -6.87670416e-20+1.12126080e-20j
   2.4991

In [144]:
## As we can see, all the checks are completely satisfied. Thus we can arrange the values of x (real values) in a diagonal density matrix and save it in a .mat file
#We will also pass in L2_redfield operator

rho_th_check = np.zeros((number,number),dtype=complex)
for i in range(number):
    for k in range(number):
        rho_th_check[i,k] = rho_th[i,k]

print(rho_th_check)

data_dict = {"dm_ness":rho_comp2, "dm_th":rho_th_new,"L2_red":L2_redfield, "dm_th_check":rho_th_check}

scipy.io.savemat(F'../matlab/ness_data_NL1 = {NL1}_NL2 = {NL2}_NM = {NM}_6.mat',data_dict)


[[5.25128563e-03+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j]
 [0.00000000e+00+0.j 1.42289418e-02+0.j 4.54597617e-05+0.j
  0.00000000e+00+0.j 7.26967645e-08+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 7.75845012e-11+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j]
 [0.00000000e+00+0.j 4.54597617e-05+0.j 1.41835547e-02+0.j
  0.00000000e+00+0.j 4.53871426e-05+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 7.26967645e-08+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.0000000

In [ ]:
## Let us then arrange the final running of the code in a block

In [37]:
NL1 = 1
NL2 = 1
NM = 2

N = NL1 + NL2 + NM
dL1 = 2**NL1
dL2 = 2**NL2
dM = 2**NM
d = 2**N
dims = [2]*N
## Define the Hamiltonian 

e = 0.01

w0list = np.array([1,1,1,1+e,1+e,1+e])
glist = np.linspace(0.016,0.016,N-1)

delta = 0

H_S = create_hamiltonian3(w0list,glist,delta,N)

create_sm_list_left = [create_sm(N,i + 1) for i in range(NL1)]
create_sm_list_right = [create_sm(N,NM + NL1 + i + 1) for i in range(NL2)]  # Create list of sigma_- operators

eigenergies,eigstates=H_S.eigenstates()
number = len(eigenergies)
## Let us compute the thermal density matrix
beta1 = 0.5 #right baths
beta2_list = np.logspace(-1,1,12)#[1.0,1.2,1.5,1.7,2.0,2.2,2.4,2.7,3.0,3.5] ###left baths
U = np.zeros((number,number),dtype=complex)
for i in range(number):
    U[:,i] = eigstates[i].full().flatten()

for index in range(len(beta2_list)):
    gamma1 = 1
    gamma2 = 1
    limit_value = 700
    b = 50
    mu1 = -1e-10
    mu2 = -1e-10
    epsilon = 0.01
    tb = 0.01
    
    beta2 = beta2_list[index]
    print("Beta2 is ",beta2)

    rho_th = (-beta2*H_S).expm()/((-beta2*H_S).expm()).tr() #left thermal density matrix
    #print(rho_th)
    number = len(eigenergies)
    integral11=np.empty((number,number),dtype=np.cdouble) #stores J * N integral for left bath
    integral12=np.empty((number,number),dtype=np.cdouble) # stores J integral (just to check) for the left bath
    integral21=np.empty((number,number),dtype=np.cdouble) #stores J*N integral for right bath
    integral22=np.empty((number,number),dtype=np.cdouble)

            #print("Integral calculations at beta2 = {} and e = {} are : ".format(beta2,e))

    for i in range(number):
        for k in range(number):
                    freq=eigenergies[k]-eigenergies[i]
                    #print(f"Absolute frequency  for i = {i}, k = {k} is ",np.absolute(freq))
                    #print(i,k,freq)
                    if( np.absolute(freq) >= 1/10**10):
                        integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta2,mu2,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                        integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #left bath done
                        integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                        integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #right bath
            
                    if (np.absolute(freq)<=1/10**10):  #The problem is arising here....
                        integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta2,mu2,gamma1),limit=limit_value)[0]
                        integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma1),limit=limit_value)[0]
                        integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value)[0]
                        integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma2),limit=limit_value)[0]
                    
                
                #expected=1.0j*(eigenergies[k]-eigenergies[i])/(2*tb*tb)
            #        print(i,k,integral2[i,k],expected)
        
        
            # PAY ATTENTION TO THE WAY THESE COEFFICIENTS ARE BEING COMPUTED
        
    constant12=np.empty((number,number),dtype=np.cdouble)
    constant11=np.empty((number,number),dtype=np.cdouble)
    constant21=np.empty((number,number),dtype=np.cdouble)
    constant22=np.empty((number,number),dtype=np.cdouble)
            
            
            
    for i in range(number):
            for k in range(number):
                    constant12[i,k]=integral12[i,k]+integral11[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma1)+func1(eigenergies[k]-eigenergies[i],tb,beta2,mu2,gamma1))    #full coefficient created this is nbar+1
                    constant11[i,k]=integral11[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta2,mu2,gamma1)                                       # the full coefficient is created
                    
                    constant22[i,k]=integral22[i,k]+integral21[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma2)+func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2))    #full coefficient created this is nbar+1
                    constant21[i,k]=integral21[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2)   # the full coefficient is created
                    #print(i,k,constant11[i,k],constant12[i,k],constant21[i,k],constant22[i,k])

    epsilon = 0.01

    ## Now we will write out the matrix elements

    A = np.zeros((number,number),dtype=complex)

    for i in range(number):
        for k in range(number):
            sum = 0
            vi = eigstates[i]
            vk = eigstates[k]
            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()
            for y in range(number):
                for l in range(NL1):
                    proj_y = eigstates[y]*eigstates[y].dag()
                    op1 = commutator(proj_k*create_sm_list_left[l]*proj_y,create_sm_list_left[l].dag())*constant11[k,y]
                    sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                    op2 = commutator(create_sm_list_left[l].dag(),proj_y*create_sm_list_left[l]*proj_k)*constant12[y,k]
                    sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

                for l in range(NL2):
                    proj_y = eigstates[y]*eigstates[y].dag()
                    op1 = commutator(proj_k*create_sm_list_right[l]*proj_y,create_sm_list_right[l].dag())*constant21[k,y]
                    sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                    op2 = commutator(create_sm_list_right[l].dag(),proj_y*create_sm_list_right[l]*proj_k)*constant22[y,k]
                    sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

            A[i,k] = sum

    b = np.zeros((number),dtype=complex)
    A_new = A[:-1]
    A_new = np.vstack([A_new,np.ones((1,number))])
    b[-1] = 1  ## Last element of b is 1

    x = np.linalg.solve(A_new,b)

    #print("Correctness check:",np.dot(A_new,x))
    #print(np.dot(A[-1],x))

    x_real = [np.real(x[i]) for i in range(number)]

    rho = np.diag(x_real)

    #set U matrix whose columns are the eigenvectors of the Hamiltonian

    

    rho_comp2 = np.dot(U,np.dot(rho,U.T.conjugate()))
    print(rho_comp2)
    L2_redfield = L2_red(rho_comp2,eigstates)

    rho_th_check = np.zeros((number,number),dtype=complex)
    for i in range(number):
        for k in range(number):
            rho_th_check[i,k] = rho_th[i,k]


    data_dict = {"dm_ness":rho_comp2,"L2_red":L2_redfield, "dm_th_check":rho_th_check, "beta2":beta2}

    scipy.io.savemat(f'../matlab/ness_data_NL1 = {NL1}_NL2 = {NL2}_NM = {NM}_{index+1}.mat',data_dict)


Beta2 is  0.1
[[ 4.37980397e-02+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  5.21056868e-02+0.j -1.24366041e-03+0.j
   0.00000000e+00+0.j  2.89511378e-04+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j -7.20147819e-05+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.24366041e-03+0.j  5.20065543e-02+0.j
   0.00000000e+00+0.j -1.22520289e-03+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  2.67006759e-04+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000